In [1]:
import bclr
import ruptures as rpt
import numpy as np
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from scipy.stats import multivariate_normal
import matplotlib.pyplot as plt

pf = PolynomialFeatures(include_bias=False)
ss = StandardScaler()

Multiple changepoint demo

In [2]:
n, dim = 500, 3  # number of samples, dimension
n_bkps, sigma = 3, 5  # number of change points, noise standard deviation
signal, bkps = rpt.pw_constant(n, dim, n_bkps, noise_std=sigma)
prior_cov=np.diag(np.repeat(1, 3))
bclrM = bclr.MultiBayesCC(signal, K=3, prior_cov=prior_cov, n_iter=1000)

In [3]:
print(bclrM.bkps)
print(bkps)

[0, 131, 240, 378, 500]
[128, 244, 378, 500]


In [4]:
bclrM.fit()

In [5]:
bclrM.transform()
[(bc.post_k_mode, bc.post_mode_prob) for bc in bclrM.bccs_]

[(130.0, 0.236), (240.0, 0.256), (378.0, 0.844)]

In [7]:
[bc.post_beta_mean for bc in bclrM.bccs_]

[array([-0.74146107,  0.58490234, -1.01334921]),
 array([ 0.70201314,  1.7993821 , -2.17075235]),
 array([-0.26350275, -0.3272134 , -0.24814657])]

In [33]:
bkps2 = [0]+bkps
for i in range(len(bkps2)-1):
    print(np.mean(signal[bkps2[i]:bkps2[i+1], :], axis=0))

[ 5.34139824 -4.85150987  4.14615732]
[ 1.83470197 -1.4062996  -1.03289697]
[  4.42049991   6.24415795 -11.07040778]
[  3.17176469   4.82550573 -11.97950142]
